In [1]:
# The code was removed by Watson Studio for sharing.

In [2]:
#For accessing files
f = open('accessfiles.py', 'wb')
f.write(streaming_body_1.read())

import accessfiles

In [1]:
!pip install pyspark==2.4.5


     |████████████████████████████████| 217.8 MB 11 kB/s  eta 0:00:017    |█████████████▍                  | 91.0 MB 9.8 MB/s eta 0:00:13     |███████████████████████▉        | 162.2 MB 9.7 MB/s eta 0:00:06     |██████████████████████████▋     | 181.0 MB 9.9 MB/s eta 0:00:04
     |████████████████████████████████| 197 kB 40.3 MB/s eta 0:00:01
  Created wheel for pyspark: filename=pyspark-2.4.5-py2.py3-none-any.whl size=218257927 sha256=12ad945a5aebe1efbabad077ee6b53ac2855faf971f2cb1938c0500bd68429e1
  Stored in directory: /tmp/wsuser/.cache/pip/wheels/01/c0/03/1c241c9c482b647d4d99412a98a5c7f87472728ad41ae55e1e
Successfully built pyspark


In [2]:
from zipfile import ZipFile
import time
import numpy as np
from operator import add

from pyspark import SparkContext, SparkConf
from pyspark.sql import SQLContext, SparkSession
from pyspark.sql.types import StructType, StructField, DoubleType, IntegerType, StringType

import os
from pyspark.sql.functions import unix_timestamp, to_date
import pyspark.sql.functions as F

sc = SparkContext.getOrCreate(SparkConf().setMaster("local[*]"))

spark = SparkSession \
    .builder \
    .getOrCreate()

spark.conf.set("spark.sql.repl.eagerEval.enabled", True)

from accessfiles import load_files

load_files()

In [3]:
movies_rdd=sc.textFile("movies.dat")
movies=movies_rdd.map(lambda lines: lines.split("::")[1])
string_flat=movies.map(lambda lines: lines.split(" ")[0])

## filter first characters for a letter then find the count
movies_letter=string_flat.filter(lambda word: word.isalpha()).map(lambda word: (word[0].lower(),1))
movies_letter_count=movies_letter.reduceByKey(lambda x,y:x+y).sortByKey()

## check for the first character for a digit then find the count
movies_digit=string_flat.filter(lambda word: word.isdigit()).map(lambda word: (word[0],1))
movies_digit_count=movies_digit.reduceByKey(lambda x,y:x+y).sortByKey()

## Union the partitions into a same file
result_rdd=movies_digit_count.union(movies_letter_count).repartition(1)

## Save the result into a pandas dataframe
result_df = result_rdd.toDF().toPandas()
result_df.columns = ['starts with','No. of movies']


In [5]:
from accessfiles import create_download_link
create_download_link(result_df, filename='Movies_starting_with.csv')